In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
from time import sleep

In [8]:
import requests

page = 0

headers = {
    'authority': 'footballapi.pulselive.com',
    'accept': '*/*',
    'accept-language': 'ru,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'if-none-match': 'W/"0747914aff6e8740e3fb239d0232b2d3d"',
    'origin': 'https://www.premierleague.com',
    'referer': 'https://www.premierleague.com/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "YaBrowser";v="23"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.962 YaBrowser/23.9.1.962 Yowser/2.5 Safari/537.36',
}

params = {
    'pageSize': '200',
    'compSeasons': '578',
    'altIds': 'true',
    'page': str(page),
    'type': 'player',
    'id': '-1',
    'compSeasonId': '578',
}

response = requests.get('https://footballapi.pulselive.com/football/players', params=params, headers=headers)

In [9]:
players_list = []
while len(response.json()['content']) > 0:
  players_list.extend(response.json()['content'])

  page += 1

  params = {
    'pageSize': '200',
    'compSeasons': '578',
    'altIds': 'true',
    'page': str(page),
    'type': 'player',
    'id': '-1',
    'compSeasonId': '578',
  }
  response = requests.get('https://footballapi.pulselive.com/football/players', params=params, headers=headers)

In [10]:

def flatten_dict(dict_, parent_key='', separator='_'):

    if type(dict_) != dict:
        raise ValueError("dict_ must be type dict")

    output_dict = {}
    for key, value in dict_.items():
        new_key = parent_key + separator + key if parent_key else key

        if isinstance(value, dict):
            nested_dict = flatten_dict(value, new_key, separator)
            output_dict.update(nested_dict)
        else:
            output_dict[new_key] = value

    return output_dict

def get_col_dict(list_dict):

    name_col_dict = {}
    count = 0

    flatten = list(map(flatten_dict, list_dict))

    for key in flatten[0].keys():
      name_col_dict[count] = key
      count += 1

    return name_col_dict


def pars_dictline(list_dict, name_col_dict):
    """ Use if each dict - line
    Input: list[dict]
    return: List[value]"""

    n = len(name_col_dict)
    flatten = list(map(flatten_dict, list_dict))
    pars_list = [[dict_feat[name_col_dict[i]] if name_col_dict[i] in dict_feat else None for i in range(n)] for dict_feat in flatten]

    return pars_list


def list_to_dict(list_dict, name_stat='name', value_stat='value', pref=''):
    """ Use if each dict - feature
    Input: list_dict: list [dict, dict]

      name_stat: key for dicts stat where are name feature
      value_stat: key for dicts stat where are value feature

    return: dict"""

    result_dict = {}

    for feat in list_dict:
        result_dict[pref+'_'+feat[name_stat]] = feat[value_stat]

    return result_dict



def pars_dictfeature(response_url, seasons, iter, main_info, stats, name_stat='name', value_stat='value'):
    """ Use if each dict - feature
    Input: response_url: url:str

          seasons: List/Series id season sorted
          iter: List/Series id club/players
          main_info: key dict response where main info
          stats: key dict where are dicts with stats whose are feature
          name_stat: key for dicts stat where are name feature
          value_stat: key for dicts stat where are value feature

    return: List[dict]"""
    result = []

    params = {
        'comps': '1',
        'compSeasons': str(seasons[0]),
        }



    for id in iter: #пробегаем по каждому клубу/игроку
        for season in seasons: #пробегаем по каждому сезону

            params = {
                'comps': '1',
                'compSeasons': str(season),
                }

            response = requests.get(f'{response_url}/{str(id)}', params=params, headers=headers)

            if response.status_code != 200:
                print(f'id: {id}, season id: {season} RESPONSE: {response.status_code}')
                continue

            if len(response.text) == 0 or len(response.json()[stats]) == 0:
                continue

            stat = response.json()[stats]
            main = response.json()[main_info]

            temp_dict = list_to_dict(stat, name_stat='name', value_stat='value')


            dict_stats = {'season_id': season, **flatten_dict(main), **temp_dict}
            result.append(dict_stats)
            sleep(0.25)
    return result


In [11]:
col = get_col_dict(players_list)
col

{0: 'playerId',
 1: 'info_position',
 2: 'info_shirtNum',
 3: 'info_positionInfo',
 4: 'nationalTeam_isoCode',
 5: 'nationalTeam_country',
 6: 'nationalTeam_demonym',
 7: 'currentTeam_name',
 8: 'currentTeam_club_name',
 9: 'currentTeam_club_abbr',
 10: 'currentTeam_club_id',
 11: 'currentTeam_teamType',
 12: 'currentTeam_shortName',
 13: 'currentTeam_id',
 14: 'currentTeam_altIds_opta',
 15: 'birth_date_millis',
 16: 'birth_date_label',
 17: 'birth_country_isoCode',
 18: 'birth_country_country',
 19: 'birth_country_demonym',
 20: 'birth_place',
 21: 'age',
 22: 'name_display',
 23: 'name_first',
 24: 'name_middle',
 25: 'name_last',
 26: 'id',
 27: 'altIds_opta'}

In [12]:
col = {0: 'playerId',
 1: 'info_position',
 2: 'info_shirtNum',
 3: 'info_positionInfo',
 4: 'nationalTeam_isoCode',
 5: 'nationalTeam_country',
 6: 'nationalTeam_demonym',
 7: 'currentTeam_name',
 8: 'currentTeam_club_name',
 9: 'currentTeam_club_abbr',
 10: 'currentTeam_club_id',
 11: 'currentTeam_teamType',
 12: 'currentTeam_shortName',
 13: 'currentTeam_id',
 14: 'currentTeam_altIds_opta',
 15: 'birth_date_millis',
 16: 'birth_date_label',
 17: 'birth_country_isoCode',
 18: 'birth_country_country',
 19: 'birth_country_demonym',
 20: 'birth_place',
 21: 'age',
 22: 'name_display',
 23: 'name_first',
 24: 'name_middle',
 25: 'name_last',
 26: 'id',
 27: 'altIds_opta'}

In [13]:
data = pars_dictline(players_list, col)
df_start_player = pd.DataFrame(data, columns=col.values())

In [14]:
df_start_player

,playerId,info_position,info_shirtNum,info_positionInfo,nationalTeam_isoCode,nationalTeam_country,nationalTeam_demonym,currentTeam_name,currentTeam_club_name,currentTeam_club_abbr,...,birth_country_country,birth_country_demonym,birth_place,age,name_display,name_first,name_middle,name_last,id,altIds_opta
0,344622.0,D,37.0,Right Full Back,GB-ENG,England,English,Bournemouth,Bournemouth,BOU,...,England,English,London,23 years 312 days,Max Aarons,Max,James,Aarons,19970.0,p232980
1,341886.0,D,NaN,Defender,GB-ENG,England,English,Nottingham Forest U21,Nottingham Forest,NFO,...,England,English,None,17 years 183 days,Zach Abbott,Zach,None,Abbott,117754.0,p549067
2,345808.0,F,12.0,Centre Striker,FI,Finland,Finnish,Carlisle United,Carlisle United,CUT,...,Ghana,Ghanan,None,22 years 31 days,Terry Ablade,Terry,None,Ablade,51536.0,p247664
3,345193.0,M,18.0,Midfielder,US,United States,American,Bournemouth,Bournemouth,BOU,...,United States,American,Wappingers Falls,24 years 271 days,Tyler Adams,Tyler,Shaan,Adams,42894.0,p200785
4,258884.0,D,4.0,Centre Central Defender,GB-ENG,England,English,Fulham,Fulham,FUL,...,England,English,None,26 years 49 days,Tosin Adarabioyo,Tosin,None,Adarabioyo,13549.0,p109646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,319999.0,D,35.0,Left Full Back,UA,Ukraine,Ukrainian,Arsenal,Arsenal,ARS,...,Ukraine,Ukrainian,Radomyshl,26 years 332 days,Oleksandr Zinchenko,Oleksandr,None,Zinchenko,19620.0,p206325
928,345780.0,M,22.0,Attacking Midfielder,MA,Morocco,Moroccan,Galatasaray,Galatasaray,GAL,...,Netherlands,Dutch,Dronten,30 years 238 days,Hakim Ziyech,Hakim,None,Ziyech,20750.0,p124183
929,288580.0,D,4.0,Centre Central Defender,FR,France,French,West Ham United,West Ham United,WHU,...,France,French,Lyon,29 years 16 days,Kurt Zouma,Kurt,Happy,Zouma,5175.0,p103192
930,267908.0,G,1.0,Goalkeeper,PL,Poland,Polish,None,None,None,...,Poland,Polish,None,19 years 137 days,Oliwier Zych,Oliwier,None,Zych,107891.0,p516159


In [15]:
df_start_player['id'] = df_start_player['id'].astype(int)
df_start_player.head(5)

,playerId,info_position,info_shirtNum,info_positionInfo,nationalTeam_isoCode,nationalTeam_country,nationalTeam_demonym,currentTeam_name,currentTeam_club_name,currentTeam_club_abbr,...,birth_country_country,birth_country_demonym,birth_place,age,name_display,name_first,name_middle,name_last,id,altIds_opta
0,344622.0,D,37.0,Right Full Back,GB-ENG,England,English,Bournemouth,Bournemouth,BOU,...,England,English,London,23 years 312 days,Max Aarons,Max,James,Aarons,19970,p232980
1,341886.0,D,NaN,Defender,GB-ENG,England,English,Nottingham Forest U21,Nottingham Forest,NFO,...,England,English,None,17 years 183 days,Zach Abbott,Zach,None,Abbott,117754,p549067
2,345808.0,F,12.0,Centre Striker,FI,Finland,Finnish,Carlisle United,Carlisle United,CUT,...,Ghana,Ghanan,None,22 years 31 days,Terry Ablade,Terry,None,Ablade,51536,p247664
3,345193.0,M,18.0,Midfielder,US,United States,American,Bournemouth,Bournemouth,BOU,...,United States,American,Wappingers Falls,24 years 271 days,Tyler Adams,Tyler,Shaan,Adams,42894,p200785
4,258884.0,D,4.0,Centre Central Defender,GB-ENG,England,English,Fulham,Fulham,FUL,...,England,English,None,26 years 49 days,Tosin Adarabioyo,Tosin,None,Adarabioyo,13549,p109646


In [16]:
df_start_player.shape

(932, 28)

In [17]:
#Парсинг всех клубов, общая инфа

params = {
    'pageSize': '400',
    'comps': '1',
    'altIds': 'true',
    'page': '0',
}

response = requests.get('https://footballapi.pulselive.com/football/teams', params=params, headers=headers)

In [18]:
club_list = response.json()['content']

In [19]:
stadium_list = []
for club in club_list: #датафрейм со стадионами

    for i in range(len(club['grounds'])):

        temp_list = [
                      club['name'] if 'name' in club else None,
                      club['id'] if 'id' in club else None,
                      club['grounds'][i]['name'] if 'name' in club['grounds'][i] else None,
                      club['grounds'][i]['id'] if 'id' in club['grounds'][i] else None,
                      club['grounds'][i]['city'] if 'city' in club['grounds'][i] else None,
                      club['grounds'][i]['capacity'] if 'capacity' in club['grounds'][i] else None,
                      club['grounds'][i]['location']['latitude'] if 'location' in club['grounds'][i] and 'latitude' in club['grounds'][i]['location'] else None,
                      club['grounds'][i]['location']['longitude'] if 'location' in club['grounds'][i] and 'longitude' in club['grounds'][i]['location'] else None,
        ]
        stadium_list.append(temp_list)

stadium_col = [
    'club',
    'club_id',
    'stadium',
    'stadium_id',
    'city',
    'capacity',
    'latitude',
    'longitude'
]

df_stadium = pd.DataFrame(stadium_list, columns=stadium_col)
df_stadium[['club_id', 'stadium_id']] = df_stadium[['club_id', 'stadium_id']].astype(int)
#df_stadium.to_csv('df_stadium.csv', index=False)

In [20]:
col = get_col_dict(club_list)
col

{0: 'name',
 1: 'club_name',
 2: 'club_shortName',
 3: 'club_abbr',
 4: 'club_id',
 5: 'teamType',
 6: 'grounds',
 7: 'shortName',
 8: 'id',
 9: 'altIds_opta'}

In [21]:
col = {
 0: 'name',
 1: 'club_name',
 2: 'club_shortName',
 3: 'club_abbr',
 4: 'club_id',
 5: 'teamType',
 6: 'grounds',
 7: 'shortName',
 8: 'id',
 9: 'altIds_opta'}

In [22]:
data = pars_dictline(club_list, col)
df_start_club = pd.DataFrame(data, columns=col.values())

In [23]:
df_start_club['club_id'] = df_start_club['club_id'].astype('int')

In [24]:
df_start_club.head()

,name,club_name,club_shortName,club_abbr,club_id,teamType,grounds,shortName,id,altIds_opta
0,Arsenal,Arsenal,Arsenal,ARS,1,FIRST,"[{'name': 'Emirates Stadium', 'city': 'London'...",Arsenal,1.0,t3
1,Aston Villa,Aston Villa,Aston Villa,AVL,2,FIRST,"[{'name': 'Villa Park', 'city': 'Birmingham', ...",Aston Villa,2.0,t7
2,Barnsley,Barnsley,Barnsley,BAR,30,FIRST,"[{'name': 'Oakwell', 'city': 'Barnsley', 'capa...",Barnsley,30.0,t37
3,Birmingham City,Birmingham City,Birmingham,BIR,35,FIRST,[{'name': 'St. Andrew's Trillion Trophy Stadiu...,Birmingham,35.0,t41
4,Blackburn Rovers,Blackburn Rovers,Blackburn,BLB,3,FIRST,"[{'name': 'Ewood Park', 'city': 'Blackburn', '...",Blackburn,3.0,t5


In [25]:
#Парсинг id сезонов

params = {
    'page': '0',
    'pageSize': '100',
}

response = requests.get('https://footballapi.pulselive.com/football/competitions/1/compseasons', params=params, headers=headers)
response

<Response [200]>

In [26]:
col = get_col_dict(response.json()['content'])
col

{0: 'label', 1: 'id'}

In [27]:
col = {0: 'label', 1: 'id'}

In [28]:
data = pars_dictline(response.json()['content'], col)
id_season = pd.DataFrame(data, columns=col.values()) 
id_season['id'] = id_season['id'].astype(int)

In [29]:
id_season

,label,id
0,2023/24,578
1,2022/23,489
2,2021/22,418
3,2020/21,363
4,2019/20,274
5,2018/19,210
6,2017/18,79
7,2016/17,54
8,2015/16,42
9,2014/15,27


In [29]:
club_id = df_start_club['club_id'].unique()
player_id = df_start_player['id'].unique()

In [30]:
r = requests.get('https://footballapi.pulselive.com/football/stats/team/1', params=params, headers=headers)
r.json()['stats']

[{'name': 'accurate_back_zone_pass',
  'value': 174838.0,
  'description': 'Todo: accurate_back_zone_pass',
  'additionalInfo': {}},
 {'name': 'accurate_chipped_pass',
  'value': 13775.0,
  'description': 'Todo: accurate_chipped_pass',
  'additionalInfo': {}},
 {'name': 'accurate_corners_intobox',
  'value': 1292.0,
  'description': 'Todo: accurate_corners_intobox',
  'additionalInfo': {}},
 {'name': 'accurate_cross',
  'value': 3871.0,
  'description': 'Todo: accurate_cross',
  'additionalInfo': {}},
 {'name': 'accurate_cross_nocorner',
  'value': 2620.0,
  'description': 'Todo: accurate_cross_nocorner',
  'additionalInfo': {}},
 {'name': 'accurate_flick_on',
  'value': 1822.0,
  'description': 'Todo: accurate_flick_on',
  'additionalInfo': {}},
 {'name': 'accurate_freekick_cross',
  'value': 252.0,
  'description': 'Todo: accurate_freekick_cross',
  'additionalInfo': {}},
 {'name': 'accurate_fwd_zone_pass',
  'value': 219860.0,
  'description': 'Todo: accurate_fwd_zone_pass',
  'addi

In [ ]:
response_url = 'https://footballapi.pulselive.com/football/stats/team'
seasons = id_season['id'].iloc[:10]
iter = club_id
main_info = 'entity'
stats = 'stats'
list_club = pars_dictfeature(response_url, seasons, iter, main_info, stats, name_stat='name', value_stat='value')

In [ ]:
col = get_col_dict(list_club)
col

{0: 'season_id',
 1: 'name',
 2: 'club_name',
 3: 'club_shortName',
 4: 'club_abbr',
 5: 'club_id',
 6: 'teamType',
 7: 'shortName',
 8: 'id',
 9: '_accurate_back_zone_pass',
 10: '_accurate_chipped_pass',
 11: '_accurate_corners_intobox',
 12: '_accurate_cross',
 13: '_accurate_cross_nocorner',
 14: '_accurate_flick_on',
 15: '_accurate_freekick_cross',
 16: '_accurate_fwd_zone_pass',
 17: '_accurate_goal_kicks',
 18: '_accurate_keeper_sweeper',
 19: '_accurate_keeper_throws',
 20: '_accurate_launches',
 21: '_accurate_layoffs',
 22: '_accurate_long_balls',
 23: '_accurate_pass',
 24: '_accurate_pull_back',
 25: '_accurate_through_ball',
 26: '_accurate_throws',
 27: '_aerial_lost',
 28: '_aerial_won',
 29: '_attempted_tackle_foul',
 30: '_attempts_conceded_ibox',
 31: '_attempts_conceded_obox',
 32: '_attempts_ibox',
 33: '_attempts_obox',
 34: '_att_assist_openplay',
 35: '_att_assist_setplay',
 36: '_att_bx_centre',
 37: '_att_bx_left',
 38: '_att_bx_right',
 39: '_att_cmiss_high',

In [ ]:
col = {0: 'season_id',
 1: 'name',
 2: 'club_name',
 3: 'club_shortName',
 4: 'club_abbr',
 5: 'club_id',
 6: 'teamType',
 7: 'shortName',
 8: 'id',
 9: '_accurate_back_zone_pass',
 10: '_accurate_chipped_pass',
 11: '_accurate_corners_intobox',
 12: '_accurate_cross',
 13: '_accurate_cross_nocorner',
 14: '_accurate_flick_on',
 15: '_accurate_freekick_cross',
 16: '_accurate_fwd_zone_pass',
 17: '_accurate_goal_kicks',
 18: '_accurate_keeper_sweeper',
 19: '_accurate_keeper_throws',
 20: '_accurate_launches',
 21: '_accurate_layoffs',
 22: '_accurate_long_balls',
 23: '_accurate_pass',
 24: '_accurate_pull_back',
 25: '_accurate_through_ball',
 26: '_accurate_throws',
 27: '_aerial_lost',
 28: '_aerial_won',
 29: '_attempted_tackle_foul',
 30: '_attempts_conceded_ibox',
 31: '_attempts_conceded_obox',
 32: '_attempts_ibox',
 33: '_attempts_obox',
 34: '_att_assist_openplay',
 35: '_att_assist_setplay',
 36: '_att_bx_centre',
 37: '_att_bx_left',
 38: '_att_bx_right',
 39: '_att_cmiss_high',
 40: '_att_cmiss_left',
 41: '_att_cmiss_right',
 42: '_att_corner',
 43: '_att_fastbreak',
 44: '_att_goal_high_centre',
 45: '_att_goal_high_left',
 46: '_att_goal_high_right',
 47: '_att_goal_low_centre',
 48: '_att_goal_low_left',
 49: '_att_goal_low_right',
 50: '_att_hd_goal',
 51: '_att_hd_miss',
 52: '_att_hd_post',
 53: '_att_hd_target',
 54: '_att_hd_total',
 55: '_att_ibox_blocked',
 56: '_att_ibox_goal',
 57: '_att_ibox_miss',
 58: '_att_ibox_post',
 59: '_att_ibox_target',
 60: '_att_lf_goal',
 61: '_att_lf_target',
 62: '_att_lf_total',
 63: '_att_miss_high',
 64: '_att_miss_high_left',
 65: '_att_miss_high_right',
 66: '_att_miss_left',
 67: '_att_miss_right',
 68: '_att_obox_blocked',
 69: '_att_obox_goal',
 70: '_att_obox_miss',
 71: '_att_obox_target',
 72: '_att_obx_centre',
 73: '_att_one_on_one',
 74: '_att_openplay',
 75: '_att_pen_goal',
 76: '_att_post_left',
 77: '_att_rf_goal',
 78: '_att_rf_target',
 79: '_att_rf_total',
 80: '_att_setpiece',
 81: '_att_sv_high_centre',
 82: '_att_sv_high_left',
 83: '_att_sv_high_right',
 84: '_att_sv_low_centre',
 85: '_att_sv_low_left',
 86: '_att_sv_low_right',
 87: '_backward_pass',
 88: '_ball_recovery',
 89: '_big_chance_created',
 90: '_big_chance_missed',
 91: '_big_chance_scored',
 92: '_blocked_cross',
 93: '_blocked_pass',
 94: '_blocked_scoring_att',
 95: '_challenge_lost',
 96: '_clean_sheet',
 97: '_clearance_off_line',
 98: '_contentious_decision',
 99: '_corner_taken',
 100: '_crosses_18yard',
 101: '_crosses_18yardplus',
 102: '_defender_goals',
 103: '_dispossessed',
 104: '_diving_save',
 105: '_draws',
 106: '_duel_lost',
 107: '_duel_won',
 108: '_effective_blocked_cross',
 109: '_effective_clearance',
 110: '_effective_head_clearance',
 111: '_error_lead_to_goal',
 112: '_error_lead_to_shot',
 113: '_final_third_entries',
 114: '_first_half_goals',
 115: '_fk_foul_lost',
 116: '_fk_foul_won',
 117: '_forward_goals',
 118: '_fouled_final_third',
 119: '_freekick_cross',
 120: '_fwd_pass',
 121: '_goals',
 122: '_goals_conceded',
 123: '_goals_conceded_ibox',
 124: '_goals_conceded_obox',
 125: '_goals_openplay',
 126: '_goal_assist',
 127: '_goal_assist_deadball',
 128: '_goal_assist_intentional',
 129: '_goal_assist_openplay',
 130: '_goal_assist_setplay',
 131: '_goal_fastbreak',
 132: '_goal_kicks',
 133: '_good_high_claim',
 134: '_hand_ball',
 135: '_head_clearance',
 136: '_hit_woodwork',
 137: '_interception',
 138: '_interceptions_in_box',
 139: '_interception_won',
 140: '_keeper_throws',
 141: '_last_man_tackle',
 142: '_leftside_pass',
 143: '_long_pass_own_to_opp',
 144: '_long_pass_own_to_opp_success',
 145: '_lost_corners',
 146: '_midfielder_goals',
 147: '_offtarget_att_assist',
 148: '_ontarget_att_assist',
 149: '_ontarget_scoring_att',
 150: '_open_play_pass',
 151: '_outfielder_block',
 152: '_overrun',
 153: '_own_goal_accrued',
 154: '_passes_left',
 155: '_passes_right',
 156: '_penalty_conceded',
 157: '_penalty_faced',
 158: '_penalty_won',
 159: '_pen_area_entries',
 160: '_pen_goals_conceded',
 161: '_possession_percentage',
 162: '_poss_lost_all',
 163: '_poss_lost_ctrl',
 164: '_poss_won_att_3rd',
 165: '_poss_won_def_3rd',
 166: '_poss_won_mid_3rd',
 167: '_post_scoring_att',
 168: '_pts_dropped_winning_pos',
 169: '_pts_gained_losing_pos',
 170: '_punches',
 171: '_put_through',
 172: '_rightside_pass',
 173: '_saved_ibox',
 174: '_saved_obox',
 175: '_saves',
 176: '_second_yellow',
 177: '_shield_ball_oop',
 178: '_shot_fastbreak',
 179: '_shot_off_target',
 180: '_six_yard_block',
 181: '_subs_goals',
 182: '_subs_made',
 183: '_successful_final_third_passes',
 184: '_successful_open_play_pass',
 185: '_successful_put_through',
 186: '_total_att_assist',
 187: '_total_back_zone_pass',
 188: '_total_chipped_pass',
 189: '_total_clearance',
 190: '_total_contest',
 191: '_total_corners_intobox',
 192: '_total_cross',
 193: '_total_cross_nocorner',
 194: '_total_fastbreak',
 195: '_total_final_third_passes',
 196: '_total_flick_on',
 197: '_total_fwd_zone_pass',
 198: '_total_high_claim',
 199: '_total_keeper_sweeper',
 200: '_total_launches',
 201: '_total_layoffs',
 202: '_total_long_balls',
 203: '_total_offside',
 204: '_total_pass',
 205: '_total_pull_back',
 206: '_total_red_card',
 207: '_total_scoring_att',
 208: '_total_tackle',
 209: '_total_through_ball',
 210: '_total_throws',
 211: '_total_yel_card',
 212: '_touches',
 213: '_touches_in_opp_box',
 214: '_unsuccessful_touch',
 215: '_wins',
 216: '_won_contest',
 217: '_won_corners',
 218: '_won_tackle',
 219: '_attendance_count',
 220: '_attendance_total',
 221: '_attendance_average',
 222: '_attendance_highest',
 223: '_attendance_lowest'}

In [ ]:
data = pars_dictline(list_club, col)
club_stat = pd.DataFrame(data, columns=col.values())

In [ ]:
club_stat.head()

,season_id,name,club_name,club_shortName,club_abbr,club_id,teamType,shortName,id,_accurate_back_zone_pass,...,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle,_attendance_count,_attendance_total,_attendance_average,_attendance_highest,_attendance_lowest
0,578,Arsenal,Arsenal,Arsenal,ARS,1.0,FIRST,Arsenal,1.0,2430.0,...,122.0,7.0,84.0,82.0,93.0,6.0,360679.0,60113.0,60233.0,59961.0
1,489,Arsenal,Arsenal,Arsenal,ARS,1.0,FIRST,Arsenal,1.0,7609.0,...,526.0,26.0,357.0,223.0,343.0,19.0,1143535.0,60186.0,60325.0,59968.0
2,418,Arsenal,Arsenal,Arsenal,ARS,1.0,FIRST,Arsenal,1.0,7337.0,...,557.0,22.0,319.0,209.0,311.0,19.0,1135743.0,59776.0,60223.0,58729.0
3,363,Arsenal,Arsenal,Arsenal,ARS,1.0,FIRST,Arsenal,1.0,8985.0,...,511.0,18.0,296.0,208.0,252.0,2.0,12000.0,6000.0,10000.0,2000.0
4,274,Arsenal,Arsenal,Arsenal,ARS,1.0,FIRST,Arsenal,1.0,8703.0,...,547.0,14.0,411.0,232.0,336.0,15.0,904188.0,60279.0,60383.0,60031.0


In [ ]:
response_url = 'https://footballapi.pulselive.com/football/stats/player'
iter = player_id
list_player = pars_dictfeature(response_url, seasons, iter, main_info, stats, name_stat='name', value_stat='value')

In [ ]:
col = get_col_dict(list_player)
col

{0: 'season_id',
 1: 'playerId',
 2: 'info_position',
 3: 'info_shirtNum',
 4: 'info_positionInfo',
 5: 'info_loan',
 6: 'nationalTeam_isoCode',
 7: 'nationalTeam_country',
 8: 'nationalTeam_demonym',
 9: 'birth_date_millis',
 10: 'birth_date_label',
 11: 'birth_country_isoCode',
 12: 'birth_country_country',
 13: 'birth_country_demonym',
 14: 'birth_place',
 15: 'age',
 16: 'name_display',
 17: 'name_first',
 18: 'name_middle',
 19: 'name_last',
 20: 'id',
 21: '_accurate_back_zone_pass',
 22: '_accurate_chipped_pass',
 23: '_accurate_cross',
 24: '_accurate_cross_nocorner',
 25: '_accurate_fwd_zone_pass',
 26: '_accurate_launches',
 27: '_accurate_long_balls',
 28: '_accurate_pass',
 29: '_accurate_through_ball',
 30: '_accurate_throws',
 31: '_aerial_lost',
 32: '_aerial_won',
 33: '_appearances',
 34: '_attempted_tackle_foul',
 35: '_attempts_conceded_ibox',
 36: '_attempts_conceded_obox',
 37: '_attempts_obox',
 38: '_att_assist_openplay',
 39: '_att_lf_total',
 40: '_att_obox_blo

In [ ]:
col = {0: 'season_id',
 1: 'playerId',
 2: 'info_position',
 3: 'info_shirtNum',
 4: 'info_positionInfo',
 5: 'info_loan',
 6: 'nationalTeam_isoCode',
 7: 'nationalTeam_country',
 8: 'nationalTeam_demonym',
 9: 'birth_date_millis',
 10: 'birth_date_label',
 11: 'birth_country_isoCode',
 12: 'birth_country_country',
 13: 'birth_country_demonym',
 14: 'birth_place',
 15: 'age',
 16: 'name_display',
 17: 'name_first',
 18: 'name_middle',
 19: 'name_last',
 20: 'id',
 21: '_accurate_back_zone_pass',
 22: '_accurate_chipped_pass',
 23: '_accurate_cross',
 24: '_accurate_cross_nocorner',
 25: '_accurate_fwd_zone_pass',
 26: '_accurate_launches',
 27: '_accurate_long_balls',
 28: '_accurate_pass',
 29: '_accurate_through_ball',
 30: '_accurate_throws',
 31: '_aerial_lost',
 32: '_aerial_won',
 33: '_appearances',
 34: '_attempted_tackle_foul',
 35: '_attempts_conceded_ibox',
 36: '_attempts_conceded_obox',
 37: '_attempts_obox',
 38: '_att_assist_openplay',
 39: '_att_lf_total',
 40: '_att_obox_blocked',
 41: '_att_obx_centre',
 42: '_att_openplay',
 43: '_backward_pass',
 44: '_ball_recovery',
 45: '_big_chance_created',
 46: '_blocked_cross',
 47: '_blocked_pass',
 48: '_blocked_scoring_att',
 49: '_challenge_lost',
 50: '_clean_sheet',
 51: '_crosses_18yard',
 52: '_crosses_18yardplus',
 53: '_dispossessed',
 54: '_draws',
 55: '_duel_lost',
 56: '_duel_won',
 57: '_effective_blocked_cross',
 58: '_effective_clearance',
 59: '_effective_head_clearance',
 60: '_final_third_entries',
 61: '_fouled_final_third',
 62: '_fouls',
 63: '_fwd_pass',
 64: '_game_started',
 65: '_goals_conceded',
 66: '_goals_conceded_ibox',
 67: '_goals_conceded_obox',
 68: '_head_clearance',
 69: '_head_pass',
 70: '_interception',
 71: '_interceptions_in_box',
 72: '_interception_won',
 73: '_leftside_pass',
 74: '_long_pass_own_to_opp',
 75: '_long_pass_own_to_opp_success',
 76: '_losses',
 77: '_lost_corners',
 78: '_mins_played',
 79: '_offside_provoked',
 80: '_offtarget_att_assist',
 81: '_ontarget_att_assist',
 82: '_open_play_pass',
 83: '_outfielder_block',
 84: '_overrun',
 85: '_passes_left',
 86: '_passes_right',
 87: '_penalty_conceded',
 88: '_pen_area_entries',
 89: '_pen_goals_conceded',
 90: '_poss_lost_all',
 91: '_poss_lost_ctrl',
 92: '_poss_won_att_3rd',
 93: '_poss_won_def_3rd',
 94: '_poss_won_mid_3rd',
 95: '_put_through',
 96: '_rightside_pass',
 97: '_successful_final_third_passes',
 98: '_successful_open_play_pass',
 99: '_successful_put_through',
 100: '_times_tackled',
 101: '_total_att_assist',
 102: '_total_back_zone_pass',
 103: '_total_chipped_pass',
 104: '_total_clearance',
 105: '_total_contest',
 106: '_total_cross',
 107: '_total_cross_nocorner',
 108: '_total_final_third_passes',
 109: '_total_flick_on',
 110: '_total_fwd_zone_pass',
 111: '_total_launches',
 112: '_total_long_balls',
 113: '_total_offside',
 114: '_total_pass',
 115: '_total_pull_back',
 116: '_total_scoring_att',
 117: '_total_sub_off',
 118: '_total_sub_on',
 119: '_total_tackle',
 120: '_total_through_ball',
 121: '_total_throws',
 122: '_touches',
 123: '_touches_in_opp_box',
 124: '_turnover',
 125: '_unsuccessful_touch',
 126: '_was_fouled',
 127: '_winning_goal',
 128: '_wins',
 129: '_won_contest',
 130: '_won_corners',
 131: '_won_tackle',
 132: '_yellow_card'}

In [ ]:
data = pars_dictline(list_player, col)
player_stat = pd.DataFrame(data, columns=col.values())

In [ ]:
player_stat.head()

,season_id,playerId,info_position,info_shirtNum,info_positionInfo,info_loan,nationalTeam_isoCode,nationalTeam_country,nationalTeam_demonym,birth_date_millis,...,_touches_in_opp_box,_turnover,_unsuccessful_touch,_was_fouled,_winning_goal,_wins,_won_contest,_won_corners,_won_tackle,_yellow_card
0,578,344622.0,D,37.0,Right Full Back,False,GB-ENG,England,English,9.469440e+11,...,7.0,9.0,9.0,22.0,0.0,1.0,8.0,2.0,12.0,1.0
1,418,344622.0,D,37.0,Right Full Back,False,GB-ENG,England,English,9.469440e+11,...,26.0,38.0,38.0,52.0,0.0,4.0,34.0,12.0,44.0,8.0
2,274,344622.0,D,37.0,Right Full Back,False,GB-ENG,England,English,9.469440e+11,...,45.0,54.0,54.0,49.0,NaN,4.0,48.0,20.0,30.0,7.0
3,489,345193.0,M,18.0,Midfielder,False,US,United States,American,9.189504e+11,...,6.0,28.0,28.0,21.0,0.0,5.0,8.0,1.0,46.0,4.0
4,578,258884.0,D,4.0,Centre Central Defender,False,GB-ENG,England,English,8.750592e+11,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
params = {
    'comps': '1',
    'compSeasons': '578',
    'pageSize': '1000',
    'altIds': 'true',
    'sort': 'desc',
    'statuses': 'C'
}


response = requests.get('https://footballapi.pulselive.com/football/fixtures',  headers=headers, params=params)
col = get_col_dict(response.json()['content'])
col

{0: 'gameweek_id',
 1: 'gameweek_compSeason_label',
 2: 'gameweek_compSeason_competition_abbreviation',
 3: 'gameweek_compSeason_competition_description',
 4: 'gameweek_compSeason_competition_level',
 5: 'gameweek_compSeason_competition_source',
 6: 'gameweek_compSeason_competition_id',
 7: 'gameweek_compSeason_competition_altIds_opta',
 8: 'gameweek_compSeason_id',
 9: 'gameweek_gameweek',
 10: 'gameweek_competitionPhase_id',
 11: 'gameweek_competitionPhase_type',
 12: 'gameweek_competitionPhase_gameweekRange',
 13: 'kickoff_completeness',
 14: 'kickoff_millis',
 15: 'kickoff_label',
 16: 'provisionalKickoff_completeness',
 17: 'provisionalKickoff_millis',
 18: 'provisionalKickoff_label',
 19: 'teams',
 20: 'replay',
 21: 'ground_name',
 22: 'ground_city',
 23: 'ground_source',
 24: 'ground_id',
 25: 'neutralGround',
 26: 'status',
 27: 'phase',
 28: 'outcome',
 29: 'attendance',
 30: 'clock_secs',
 31: 'clock_label',
 32: 'fixtureType',
 33: 'extraTime',
 34: 'shootout',
 35: 'goals'

In [ ]:
col = {0: 'gameweek_id',
 1: 'gameweek_compSeason_label',
 2: 'gameweek_compSeason_competition_abbreviation',
 3: 'gameweek_compSeason_competition_description',
 4: 'gameweek_compSeason_competition_level',
 5: 'gameweek_compSeason_competition_source',
 6: 'gameweek_compSeason_competition_id',
 7: 'gameweek_compSeason_competition_altIds_opta',
 8: 'gameweek_compSeason_id',
 9: 'gameweek_gameweek',
 10: 'gameweek_competitionPhase_id',
 11: 'gameweek_competitionPhase_type',
 12: 'gameweek_competitionPhase_gameweekRange',
 13: 'kickoff_completeness',
 14: 'kickoff_millis',
 15: 'kickoff_label',
 16: 'provisionalKickoff_completeness',
 17: 'provisionalKickoff_millis',
 18: 'provisionalKickoff_label',
 19: 'teams',
 20: 'replay',
 21: 'ground_name',
 22: 'ground_city',
 23: 'ground_source',
 24: 'ground_id',
 25: 'neutralGround',
 26: 'status',
 27: 'phase',
 28: 'outcome',
 29: 'attendance',
 30: 'clock_secs',
 31: 'clock_label',
 32: 'fixtureType',
 33: 'extraTime',
 34: 'shootout',
 35: 'goals',
 36: 'penaltyShootouts',
 37: 'behindClosedDoors',
 38: 'id',
 39: 'altIds_opta'}

In [ ]:
df_games = pd.DataFrame()

for season in id_season['id'].iloc[:10]:

    params['compSeasons'] = season
    response = requests.get('https://footballapi.pulselive.com/football/fixtures',  headers=headers, params=params)
    r = response.json()['content']
    value = pars_dictline(r, col)

    temp_df = pd.DataFrame(value, columns=col.values())

    df_games = pd.concat([df_games, temp_df], ignore_index=True)
    sleep(1)


In [ ]:
df_games.shape

(3520, 40)

In [ ]:
df_games

,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_competition_abbreviation,gameweek_compSeason_competition_description,gameweek_compSeason_competition_level,gameweek_compSeason_competition_source,gameweek_compSeason_competition_id,gameweek_compSeason_competition_altIds_opta,gameweek_compSeason_id,gameweek_gameweek,...,clock_secs,clock_label,fixtureType,extraTime,shootout,goals,penaltyShootouts,behindClosedDoors,id,altIds_opta
0,12278.0,2023/24,EN_PR,Premier League,SEN,,1.0,8,578.0,10.0,...,5760.0,90 +6'00,REGULAR,False,False,"[{'personId': 65970.0, 'clock': {'secs': 1560....",[],False,93418.0,g2367635
1,12278.0,2023/24,EN_PR,Premier League,SEN,,1.0,8,578.0,10.0,...,5700.0,90 +5'00,REGULAR,False,False,"[{'personId': 51423.0, 'assistId': 24626.0, 'c...",[],False,93412.0,g2367630
2,12278.0,2023/24,EN_PR,Premier League,SEN,,1.0,8,578.0,10.0,...,6000.0,90 +10'00,REGULAR,False,False,"[{'personId': 71590.0, 'assistId': 22542.0, 'c...",[],False,93414.0,g2367631
3,12278.0,2023/24,EN_PR,Premier League,SEN,,1.0,8,578.0,10.0,...,5700.0,90 +5'00,REGULAR,False,False,"[{'personId': 19758.0, 'clock': {'secs': 1860....",[],False,93417.0,g2367634
4,12278.0,2023/24,EN_PR,Premier League,SEN,,1.0,8,578.0,10.0,...,5820.0,90 +7'00,REGULAR,False,False,"[{'personId': 9576.0, 'assistId': 33135.0, 'cl...",[],False,93419.0,g2367636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,900.0,2014/15,EN_PR,Premier League,SEN,,1.0,8,27.0,1.0,...,5760.0,90 +6'00,REGULAR,False,False,"[{'personId': 2936.0, 'assistId': 3225.0, 'clo...",[],False,9614.0,g755309
3516,900.0,2014/15,EN_PR,Premier League,SEN,,1.0,8,27.0,1.0,...,5760.0,90 +6'00,REGULAR,False,False,"[{'personId': 3803.0, 'clock': {'secs': 3000.0...",[],False,9615.0,g755310
3517,900.0,2014/15,EN_PR,Premier League,SEN,,1.0,8,27.0,1.0,...,5760.0,90 +6'00,REGULAR,False,False,"[{'personId': 4205.0, 'clock': {'secs': 2520.0...",[],False,9616.0,g755311
3518,900.0,2014/15,EN_PR,Premier League,SEN,,1.0,8,27.0,1.0,...,5760.0,90 +6'00,REGULAR,False,False,"[{'personId': 4112.0, 'assistId': 3960.0, 'clo...",[],False,9617.0,g755312


In [ ]:
df_games.columns

Index(['gameweek_id', 'gameweek_compSeason_label',
       'gameweek_compSeason_competition_abbreviation',
       'gameweek_compSeason_competition_description',
       'gameweek_compSeason_competition_level',
       'gameweek_compSeason_competition_source',
       'gameweek_compSeason_competition_id',
       'gameweek_compSeason_competition_altIds_opta', 'gameweek_compSeason_id',
       'gameweek_gameweek', 'gameweek_competitionPhase_id',
       'gameweek_competitionPhase_type',
       'gameweek_competitionPhase_gameweekRange', 'kickoff_completeness',
       'kickoff_millis', 'kickoff_label', 'provisionalKickoff_completeness',
       'provisionalKickoff_millis', 'provisionalKickoff_label', 'teams',
       'replay', 'ground_name', 'ground_city', 'ground_source', 'ground_id',
       'neutralGround', 'status', 'phase', 'outcome', 'attendance',
       'clock_secs', 'clock_label', 'fixtureType', 'extraTime', 'shootout',
       'goals', 'penaltyShootouts', 'behindClosedDoors', 'id', 'altIds_o

In [ ]:
df_games['id'].astype(int)

0       93418
1       93412
2       93414
3       93417
4       93419
        ...  
3515     9614
3516     9615
3517     9616
3518     9617
3519     9613
Name: id, Length: 3520, dtype: int64

In [ ]:
response = requests.get('https://footballapi.pulselive.com/football/stats/match/93392', headers=headers)


In [ ]:
response.json().keys()

dict_keys(['entity', 'data'])

In [ ]:
response.json()['entity'].keys()

dict_keys(['gameweek', 'kickoff', 'provisionalKickoff', 'teams', 'replay', 'ground', 'neutralGround', 'status', 'phase', 'outcome', 'attendance', 'clock', 'fixtureType', 'extraTime', 'shootout', 'behindClosedDoors', 'id'])

In [ ]:
for i in response.json()['entity'].keys():
    print(f"{i}: {response.json()['entity'][i]}")
    print()

gameweek: {'id': 12276, 'compSeason': {'label': '2023/24', 'competition': {'abbreviation': 'EN_PR', 'description': 'Premier League', 'level': 'SEN', 'source': '', 'id': 1}, 'id': 578}, 'gameweek': 8}

kickoff: {'completeness': 3, 'millis': 1696770000000, 'label': 'Sun 8 Oct 2023, 14:00 BST', 'gmtOffset': 1.0}

provisionalKickoff: {'completeness': 3, 'millis': 1696770000000, 'label': 'Sun 8 Oct 2023, 14:00 BST', 'gmtOffset': 1.0}

teams: [{'team': {'name': 'Brighton & Hove Albion', 'club': {'name': 'Brighton and Hove Albion', 'shortName': 'Brighton', 'abbr': 'BHA', 'id': 131}, 'teamType': 'FIRST', 'shortName': 'Brighton', 'id': 131}, 'score': 2}, {'team': {'name': 'Liverpool', 'club': {'name': 'Liverpool', 'shortName': 'Liverpool', 'abbr': 'LIV', 'id': 10}, 'teamType': 'FIRST', 'shortName': 'Liverpool', 'id': 10}, 'score': 2}]

replay: False

ground: {'name': 'Amex Stadium', 'city': 'Falmer', 'source': 'OPTA', 'id': 1684}

neutralGround: False

status: C

phase: F

outcome: D

attendanc

In [ ]:
r = response.json()
r

{'entity': {'gameweek': {'id': 12276,
   'compSeason': {'label': '2023/24',
    'competition': {'abbreviation': 'EN_PR',
     'description': 'Premier League',
     'level': 'SEN',
     'source': '',
     'id': 1},
    'id': 578},
   'gameweek': 8},
  'kickoff': {'completeness': 3,
   'millis': 1696770000000,
   'label': 'Sun 8 Oct 2023, 14:00 BST',
   'gmtOffset': 1.0},
  'provisionalKickoff': {'completeness': 3,
   'millis': 1696770000000,
   'label': 'Sun 8 Oct 2023, 14:00 BST',
   'gmtOffset': 1.0},
  'teams': [{'team': {'name': 'Brighton & Hove Albion',
     'club': {'name': 'Brighton and Hove Albion',
      'shortName': 'Brighton',
      'abbr': 'BHA',
      'id': 131},
     'teamType': 'FIRST',
     'shortName': 'Brighton',
     'id': 131},
    'score': 2},
   {'team': {'name': 'Liverpool',
     'club': {'name': 'Liverpool',
      'shortName': 'Liverpool',
      'abbr': 'LIV',
      'id': 10},
     'teamType': 'FIRST',
     'shortName': 'Liverpool',
     'id': 10},
    'score': 2

In [ ]:
r['entity']['teams'][0]['team_1'] = r['entity']['teams'][0]['team']
r['entity']['teams'][1]['team_2'] = r['entity']['teams'][1]['team']

r['entity']['teams'][0]['score_1'] = r['entity']['teams'][0]['score']
r['entity']['teams'][1]['score_2'] = r['entity']['teams'][1]['score']

del r['entity']['teams'][0]['team']
del r['entity']['teams'][1]['team']

del r['entity']['teams'][0]['score']
del r['entity']['teams'][1]['score']

In [ ]:
r['entity']['teams'] = flatten_dict(r['entity']['teams'][0]) | flatten_dict(r['entity']['teams'][1])

In [ ]:
match_id = df_games['id'].astype(int)
url = 'https://footballapi.pulselive.com/football/stats/match'

In [ ]:
result = []
s = requests.Session()
count = 0
for id in match_id: #пробегаем по каждому клубу/игроку
    count += 1
    r = s.get(f'{url}/{str(id)}', headers=headers)

    if r.status_code != 200:
        print(f'match id:: {id}, RESPONSE: {r.status_code}')
        continue

    if len(r.text) == 0 or len(r.json()['entity']) == 0 or len(r.json()['data']) == 0:
        continue

    r = r.json()
    r['entity']['teams'][0]['team_1'] = r['entity']['teams'][0]['team']
    r['entity']['teams'][1]['team_2'] = r['entity']['teams'][1]['team']

    r['entity']['teams'][0]['score_1'] = r['entity']['teams'][0]['score']
    r['entity']['teams'][1]['score_2'] = r['entity']['teams'][1]['score']

    del r['entity']['teams'][0]['team']
    del r['entity']['teams'][1]['team']

    del r['entity']['teams'][0]['score']
    del r['entity']['teams'][1]['score']

    r['entity']['teams'] = flatten_dict(r['entity']['teams'][0]) | flatten_dict(r['entity']['teams'][1])

    main = flatten_dict(r['entity'])

    team_1_id = str(main['teams_team_1_id'])
    team_2_id = str(main['teams_team_2_id'])

    team_1_stat = r['data'][team_1_id]['M']
    team_2_stat = r['data'][team_2_id]['M']

    team_1_stat_dict = list_to_dict(team_1_stat, name_stat='name', value_stat='value', pref='team_1')
    team_2_stat_dict = list_to_dict(team_2_stat, name_stat='name', value_stat='value', pref='team_2')


    dict_stats = {'match_id': id, **flatten_dict(main), **team_1_stat_dict, **team_2_stat_dict}
    sleep(0.35)
    if count % 100 == 0:
        print(count)
    result.append(dict_stats)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [ ]:
pd.options.display.max_colwidth = None

In [ ]:
col = get_col_dict(result)

In [ ]:
data = pars_dictline(result, col)
df_main = pd.DataFrame(data, columns=col.values())
df_main.head()

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_competition_abbreviation,gameweek_compSeason_competition_description,gameweek_compSeason_competition_level,gameweek_compSeason_competition_source,gameweek_compSeason_competition_id,gameweek_compSeason_id,gameweek_gameweek,...,team_2_att_lf_target,team_2_att_sv_low_centre,team_2_accurate_through_ball,team_2_midfielder_goals,team_2_att_goal_low_left,team_2_att_miss_left,team_2_outfielder_block,team_2_subs_made,team_2_clean_sheet,team_2_wins
0,93418,12278,2023/24,EN_PR,Premier League,SEN,,1,578,10,...,3.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
1,93412,12278,2023/24,EN_PR,Premier League,SEN,,1,578,10,...,NaN,1.0,1.0,NaN,NaN,NaN,6.0,5.0,NaN,NaN
2,93414,12278,2023/24,EN_PR,Premier League,SEN,,1,578,10,...,1.0,2.0,NaN,1.0,NaN,3.0,4.0,5.0,NaN,NaN
3,93417,12278,2023/24,EN_PR,Premier League,SEN,,1,578,10,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN
4,93419,12278,2023/24,EN_PR,Premier League,SEN,,1,578,10,...,2.0,1.0,NaN,NaN,1.0,1.0,2.0,1.0,1.0,1.0


In [ ]:
df_main.to_csv('df_main.csv', index=False)

In [ ]:
r['entity']

{'gameweek': {'id': 900,
  'compSeason': {'label': '2014/15',
   'competition': {'abbreviation': 'EN_PR',
    'description': 'Premier League',
    'level': 'SEN',
    'source': '',
    'id': 1},
   'id': 27},
  'gameweek': 1},
 'kickoff': {'completeness': 3,
  'millis': 1408189500000,
  'label': 'Sat 16 Aug 2014, 12:45 BST',
  'gmtOffset': 1.0},
 'provisionalKickoff': {'completeness': 3,
  'millis': 1408189500000,
  'label': 'Sat 16 Aug 2014, 12:45 BST',
  'gmtOffset': 1.0},
 'teams': {'team_1_name': 'Manchester United',
  'team_1_club_name': 'Manchester United',
  'team_1_club_shortName': 'Man Utd',
  'team_1_club_abbr': 'MUN',
  'team_1_club_id': 12,
  'team_1_teamType': 'FIRST',
  'team_1_shortName': 'Man Utd',
  'team_1_id': 12,
  'score_1': 1,
  'team_2_name': 'Swansea City',
  'team_2_club_name': 'Swansea City',
  'team_2_club_shortName': 'Swansea',
  'team_2_club_abbr': 'SWA',
  'team_2_club_id': 45,
  'team_2_teamType': 'FIRST',
  'team_2_shortName': 'Swansea',
  'team_2_id': 4

In [ ]:
flatten_dict(r['entity'])

{'gameweek_id': 900,
 'gameweek_compSeason_label': '2014/15',
 'gameweek_compSeason_competition_abbreviation': 'EN_PR',
 'gameweek_compSeason_competition_description': 'Premier League',
 'gameweek_compSeason_competition_level': 'SEN',
 'gameweek_compSeason_competition_source': '',
 'gameweek_compSeason_competition_id': 1,
 'gameweek_compSeason_id': 27,
 'gameweek_gameweek': 1,
 'kickoff_completeness': 3,
 'kickoff_millis': 1408189500000,
 'kickoff_label': 'Sat 16 Aug 2014, 12:45 BST',
 'kickoff_gmtOffset': 1.0,
 'provisionalKickoff_completeness': 3,
 'provisionalKickoff_millis': 1408189500000,
 'provisionalKickoff_label': 'Sat 16 Aug 2014, 12:45 BST',
 'provisionalKickoff_gmtOffset': 1.0,
 'teams_team_1_name': 'Manchester United',
 'teams_team_1_club_name': 'Manchester United',
 'teams_team_1_club_shortName': 'Man Utd',
 'teams_team_1_club_abbr': 'MUN',
 'teams_team_1_club_id': 12,
 'teams_team_1_teamType': 'FIRST',
 'teams_team_1_shortName': 'Man Utd',
 'teams_team_1_id': 12,
 'teams_

In [ ]:
response.json()['data']['131']['M']

[{'name': 'formation_used',
  'value': 4231.0,
  'description': 'Todo: formation_used',
  'additionalInfo': {}},
 {'name': 'attempted_tackle_foul',
  'value': 7.0,
  'description': 'Todo: attempted_tackle_foul',
  'additionalInfo': {}},
 {'name': 'dispossessed',
  'value': 10.0,
  'description': 'Todo: dispossessed',
  'additionalInfo': {}},
 {'name': 'touches',
  'value': 611.0,
  'description': 'Todo: touches',
  'additionalInfo': {}},
 {'name': 'total_back_zone_pass',
  'value': 276.0,
  'description': 'Todo: total_back_zone_pass',
  'additionalInfo': {}},
 {'name': 'fwd_pass',
  'value': 129.0,
  'description': 'Todo: fwd_pass',
  'additionalInfo': {}},
 {'name': 'possession_percentage',
  'value': 46.0,
  'description': 'Todo: possession_percentage',
  'additionalInfo': {}},
 {'name': 'backward_pass',
  'value': 73.0,
  'description': 'Todo: backward_pass',
  'additionalInfo': {}},
 {'name': 'poss_won_def_3rd',
  'value': 24.0,
  'description': 'Todo: poss_won_def_3rd',
  'additio

In [ ]:
"""df_start_player.to_csv('df_start_player.csv', index=False)
df_start_club.to_csv('df_start_club.csv', index=False)
club_stat.to_csv('club_stat.csv', index=False)
player_stat.to_csv('player_stat.csv', index=False)
df_games.to_csv('df_games.csv', index=False)"""

"df_start_player.to_csv('df_start_player.csv', index=False)\ndf_start_club.to_csv('df_start_club.csv', index=False)\nclub_stat.to_csv('club_stat.csv', index=False)\nplayer_stat.to_csv('player_stat.csv', index=False)\ndf_games.to_csv('df_games.csv', index=False)"

In [ ]:
print(f'df_start_player - {df_start_player.shape}\
      \ndf_start_club - {df_start_club.shape}\nclub_stat - {club_stat.shape}\
      \nplayer_stat - {player_stat.shape}\ndf_games - {df_games.shape}')

df_start_player - (929, 28)      
df_start_club - (51, 10)
club_stat - (200, 224)      
player_stat - (2390, 133)
df_games - (3515, 42)


In [ ]:
df_games_full = pd.DataFrame()

for id_game in df_games['id'].astype(int):

    response = requests.get(f'https://footballapi.pulselive.com/football/stats/match/{id_game}',  headers=headers)
    r = response.json()
    value, col = pars_dictline(r)

    temp_df = pd.DataFrame(value, columns=col)

    df_games_full = pd.concat([df_games_full, temp_df], ignore_index=True)
    sleep(1)

ValueError: ignored

In [ ]:
response_url = 'https://footballapi.pulselive.com/football/stats/match'
seasons = [0]
iter = df_games['id'].astype(int)
main_info = 'entity'
stats = 'stats'
list_club = pars_dictfeature(response_url, seasons, iter, main_info, stats, name_stat='name', value_stat='value')

In [ ]:
response.json()['content']